In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pprint
import pandas as pd

pp = pprint.PrettyPrinter()

In [2]:
# To do:
# Build a get segments and segment IDs within a view function


# General functions
def get_service_client(scopes, api_name, api_version, service_key_file=None, credentials=None):
    
    if credentials:
        service_client = build(api_name, api_version, credentials=credentials)
    
    else:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(service_key_file, scopes=scopes)
        service_client = build(api_name, api_version, credentials=credentials)
    
    return service_client

# Management API functions
def print_accounts(account_summary):
    for account in account_summary.get('items'):
        print('\nAccount:', account.get('name'))
        for prop in account.get('webProperties'):
            print('\tProperty:', prop.get('name'))
            for view in prop.get('profiles'):
                print('\t\tView:', view.get('name'), ' - ', view.get('id'))

def get_account_summary(management_client, show_accounts=None):
    """
    Requires that you pass a service client built on management API
    If you specify show_accounts=True, will log out account structure from account_summary object
    """
    account_summary = management_client.management().accountSummaries().list().execute()
    account_summary.keys()
    
    if show_accounts:
        print_accounts(account_summary)
    
    return account_summary

# Reporting API functions
def build_request_parameters(view_id, start_date, end_date, sampling_level=None, segments=None, cohorts=None):
    # Will need to update this once we get around to adding segments, etc.
    request_parameters = {'dateRanges': [{'startDate': start_date,
                                          'endDate': end_date}],
                          'viewId': view_id}
    
    if sampling_level:
        request_parameters.update({"samplingLevel": sampling_level})

    return request_parameters

def build_request_body(data_requests):
    
    request_body = {'reportRequests': []}
    for request in data_requests:
        
        formatted_request = {'metrics': [],
                            'dimensions': []}
        
        for metric in request.get('metrics'):
            formatted_request['metrics'].append({'expression': 'ga:'+ metric})
        
        for dimension in request.get('dimensions'):
            formatted_request['dimensions'].append({'name': 'ga:'+ dimension})
            
        request_body['reportRequests'].append(formatted_request)
    
    return request_body

def get_analytics_data(reporting_client, request_parameters, request_body):

    for request in request_body['reportRequests']:
        request.update(request_parameters)

    response_data = reporting_client.reports().batchGet(
        body=request_body
        ).execute()

    return response_data

def response_to_dataframe(response_data):
    
    data_frames = []
    
    # Yank out the meta data for each report
    for report in response_data.get('reports', []):
        headers = report.get('columnHeader', {})
        dimension_headers = headers.get('dimensions', [])
        metric_details = headers.get('metricHeader', {}).get('metricHeaderEntries', [])
        
        # Run through to grab metric headers
        metric_headers = []
        for metric in metric_details:
            metric_headers.append(metric.get('name'))
        
        # Pull dimension headers and metric headers together
        column_headers = tuple(dimension_headers + metric_headers)
        
        # For each report, yank out the actual data values
        rows_list = []
        for row in report.get('data').get('rows'):
            
            # Creates a tuple for each row of raw data
            row_data = []
            for dim_value in row.get('dimensions'):
                row_data.append(dim_value)

            for metric_value in row.get('metrics')[0].get('values'):
                row_data.append(metric_value)

            rows_list.append(tuple(row_data))

        # Build a data frame out of the data and append to list for multiple reports
        df = pd.DataFrame(rows_list, columns=column_headers)
        data_frames.append(df)
    
    return data_frames

def remove_ga_prefix(string):
    result = string.split(':')[1]
    return result

def clean_strings(strings, operations):
    results = []
    for string in strings:
        for operation in clean_operations:
            string = operation(string)
        results.append(string)
    return results

In [3]:
scopes = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/analytics.edit']
service_credentials_file = 'client_secrets.json'

In [7]:
management_client = get_service_client(scopes, 'analytics', 'v3', service_credentials_file)
account_summary = get_account_summary(management_client, show_accounts=True)


Account: API Test Account
	Property: API Test Property
		View: All Web Site Data  -  170438985

Account: jordanmoritz.com
	Property: apiTest
		View: apiTest View  -  167372989
	Property: apiTest 3rd View
		View: All Web Site Data  -  170888352
	Property: jordanmoritz.com
		View: All Web Site Data - Production  -  95100800

Account: Third API Test Account
	Property: API Test Three
		View: All Web Site Data  -  170953983
	Property: API Test Three - Second Prop
		View: All Web Site Data  -  171012332


In [ ]:
def get_view_ids(account_summary):
    view_ids = []
    for account in account_summary.get('items'):
        print('\nAccount:', account.get('name'))
        for prop in account.get('webProperties'):
            print('\tProperty:', prop.get('name'))
            for view in prop.get('profiles'):
                print('\t\tView:', view.get('name'), ' - ', view.get('id'))

In [16]:
accounts = [prop.get('name') for prop in account.get('webProperties') for account in account_summary.get('items')]
accounts

NameError: name 'account' is not defined

In [23]:
accounts = [prop.get('name') for prop in account.get('webProperties') for account in account_summary.get('items')]
accounts

NameError: name 'account' is not defined

In [27]:
#   l a b c d e f
#   ↓ ↓ ↓ ↓ ↓ ↓ ↓
l = [ [ [ [ [ [ 1 ] ] ] ] ] ]
for a in l:
    for b in a:
        for c in b:
            for d in c:
                for e in d:
                    for f in e:
                        print(float(f))

[float(f)
    for a in l
        for b in a
            for c in b
                for d in c
                    for e in d
                        for f in e]

#Which can be written in single line as
[float(f) for a in l for b in a for c in b for d in c for e in d for f in e]

# So it goes, inner most thing for second outer most for outermoster, 

[float(f) for a in l for b in a for c in b for d in c for e in d for f in e]

1.0


[1.0]

In [29]:
account_summary.get('items')

[{'id': '114693333',
  'kind': 'analytics#accountSummary',
  'name': 'API Test Account',
  'webProperties': [{'id': 'UA-114693333-1',
    'internalWebPropertyId': '170577438',
    'kind': 'analytics#webPropertySummary',
    'level': 'STANDARD',
    'name': 'API Test Property',
    'profiles': [{'id': '170438985',
      'kind': 'analytics#profileSummary',
      'name': 'All Web Site Data',
      'type': 'WEB'}],
    'websiteUrl': 'https://jordanmoritz.com'}]},
 {'id': '57614965',
  'kind': 'analytics#accountSummary',
  'name': 'jordanmoritz.com',
  'webProperties': [{'id': 'UA-57614965-2',
    'internalWebPropertyId': '167086735',
    'kind': 'analytics#webPropertySummary',
    'level': 'STANDARD',
    'name': 'apiTest',
    'profiles': [{'id': '167372989',
      'kind': 'analytics#profileSummary',
      'name': 'apiTest View',
      'type': 'WEB'}],
    'websiteUrl': 'https://jordanmoritz.com'},
   {'id': 'UA-57614965-3',
    'internalWebPropertyId': '171195104',
    'kind': 'analytics

In [ ]:
#  Accounts > Props > Views
# And we want views

# so it's views.name for props in accounts

In [ ]:
views = [view.get('name') for prop.get('profiles') in account.get('webProperties') ]

In [13]:
views = [view.get('name') for view in prop.get('profiles') for prop in account.get('webProperties') for account in account_summary.get('items')]
views

NameError: name 'prop' is not defined

In [ ]:
view_id = '95100800'
start_date = '28daysAgo'
end_date = 'today'

data_requests = [
    {'metrics': ['sessions', 'pageviews'],
     'dimensions': ['country', 'browser']},
    {'metrics': ['sessions', 'pageviews', 'bounces'],
     'dimensions': ['fullReferrer', 'country', 'browser']}]

clean_operations = [str.strip, remove_ga_prefix, str.title]

In [5]:
reporting_client = get_service_client(scopes, 'analyticsreporting', 'v4', service_credentials_file)
request_body = build_request_body(data_requests)
request_parameters = build_request_parameters(view_id, start_date, end_date)
response_data = get_analytics_data(reporting_client, request_parameters, request_body)
analytics_df = response_to_dataframe(response_data)

for df in analytics_df:
    df.columns = clean_strings(df.columns, clean_operations)

In [6]:
analytics_df[0]

,Country,Browser,Sessions,Pageviews
0,Brazil,Chrome,1,1
1,Germany,Internet Explorer,1,1
2,Italy,Safari,1,1
3,Philippines,Chrome,1,1
4,United States,Chrome,4,10
